## Here we import the basic libraries.

In particular, in order to use "Spectral_tools", this has to be imported from the same
folder.

In [7]:
import sys
!{sys.executable} -m pip install import_ipynb

In [8]:
## Importation of basic rotutine
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import ticker, cm
from scipy import linalg as LA
from math import sin, cos, pi, log
from math import tanh
import cmath
import import_ipynb

## Importation of Spectral tools
from Spectral_tools import * #D1_CL, D2_CL,matrice_random, norm_2, Pseudospectrum



# use LaTeX fonts in the plot
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

## Main part of the code

1. We import the Chebyshev grid. We can choose between Chebyshev-Gauss (CG) or Chebyshev-Lobatto(CL). The first one does not get to the boundaries of the domain, wheras the second one does. 

2.Introduce the operator for which we want to calculate the eigenvalues.
This includes:

a) The choice of potential: this comes from our problem and there is a "constant" term,
a $tanh$ terms and a $sech^2$ term.

b) A random matrix perturbation.

3. Calculation of eigenvalues (QNMs) and Pseudospectrum is done.


In [9]:

# Number of points n= N+1
#N = 100
def Matrix_Eigenvalues_Test(N, epsilon_Random_V):

    # Spectral Differentiation, identity and Zero matrices
    #xt = [D1_CL(N)[0]]
    xt = [D1_CG(N)[0]]
    x = np.transpose(xt)
    #D1 = D1_CL(N)[1]
    #D2 = D2_CL(N)[1]
    D1 = D1_CG(N)[1]
    D2 = D2_CG(N)[1]
    Id = np.eye(N+1)
    Id_total = np.eye(2*(N+1))
    Zero = np.zeros((N+1,N+1))

    ### Matrix L definition
    #L = D1
    #LT = L.transpose()
    #LTL = np.dot(LT,L)
    #L_red = L[1:N+1,1:N+1]
    
     # Potential
    aa=2.
    V_1=40.
    V_2=40.
    V = -1. + V_1*(1-(tanh(aa))**2.)/(1+x*tanh(aa))**2. + V_2*(1-(tanh(aa))**2.)/(1-x*tanh(aa))**2.
    epsilon1 = 0.#1e-6
    epsilon2 = epsilon_Random_V
    epsilon3 = 0.#1e-6 
    NN=20
    V = -80. + V_1*(1-(math.tanh(aa))**2.)/(1+x*math.tanh(aa))**2. + V_2*(1-(math.tanh(aa))**2.)/(1-x*math.tanh(aa))**2. + epsilon1*np.cos((2*NN+1)/2.*pi*x)
    #V_random = matrice_random(N+1)
    V_random = Diagonal_Random_Matrix(N+1) - 0.5*Id
    V_random = V_random/norm_2(V_random)
    V_random = epsilon2 * V_random
    visualisation_matrix(V_random)
    print("Random matrix : ", V_random)
    norm_V_random = norm_2(V_random)+1e-14
    #V_random = epsilon2*matrice_random(N+1)
    print("L2-norm of Random_Matrix = " ,  norm_V_random, "\n")
    print("log_10 (L2-norm of Random_Matrix) = " , log(norm_V_random,10), "\n")
    
    V_random_total = epsilon3*matrice_random(2*(N+1))

    # Elementary Operators Version 1
    #lpha, beta, gamma = 0., 0., 1.
    #print("alpha =", alpha, ", beta =", beta, ", gamma =", gamma)
    #V = alpha/(1-x**2) + beta*x/(1-x**2) + gamma + V_random/(1-x**2)
    #V = 1.
    #L1 = (1-x**2)*D2 - 2*x*D1 - V*Id #+ V_random
    #L2 = -2*x*D1 - Id
    #V_random_vector = np.diag(V_random)
    #DV_random = np.dot(D1,V_random_vector)
    #L2 = -2*(x+V_random_vector)*D1 - (Id + DV_random)
    #B = Id_total
    
    # Operator matrix
    #Lu = np.concatenate((Zero,Id), axis=1)
    #Ld = np.concatenate((L1,L2), axis=1)
    #L = np.concatenate((Lu,Ld), axis=0)
    #L += epsilon3*V_random_total
    
    # Elementary Operators Version 2
    alpha, beta, gamma = 0., 0, V
    print("alpha =", alpha, ", beta =", beta, ", gamma =", gamma)
    V = alpha + beta*x + gamma*(1-x**2) + V_random
    L1 = (1-x**2)*((1-x**2)*D2 - 2*x*D1) - V*Id #+ V_random
    L2 = (1-x**2)*(-2*x*D1 - Id)
    Bu = np.concatenate((Id,Zero), axis=1)
    Bd = np.concatenate((Zero,(1-x**2)*Id), axis=1)
    B = np.concatenate((Bu,Bd), axis=0)
    #print("B", B)
    
    # Operator matrix
    L = [Zero, Id]
    Lu = np.concatenate((Zero,Id), axis=1)
    Ld = np.concatenate((L1,L2), axis=1)
    L = np.concatenate((Lu,Ld), axis=0)
    # += epsilon3*V_random_total

    #print(L1)
    #print(L2)
    
     
    L = -1j*L
    
    
    ## Pseudospectrum parameters
    ############################
    # Grid
    xmin, xmax ,ymin,ymax, Nxgrid, Nygrid = -40., 40., -10., 60., 50, 50
    #xmin, xmax ,ymin,ymax, Nxgrid, Nygrid = -40., 40., -10., 60., 100, 100
    #xmin, xmax ,ymin,ymax, Nxgrid, Nygrid = 0.86, 0.87, 0.495, 0.505, 50, 50
    #xmin, xmax ,ymin,ymax, Nxgrid, Nygrid = -2., 2., -1., 20., 50, 50
    # Heights
    #heights = np.arange(0.0001, 0.001, 0.00025)
    heights = [1e-15,1e-14,1e-13,1e-12,1e-11,1e-10,1e-9,1e-7,1e-5,1e-3,1e-1,1e0, 1e1]
    print("heights =", heights)
    # "Filled" (set fl to "f") or "Contour" (set fl to something different to "")
    fl = "f"
    print("fl = ", fl, "\n")
    #Pseudospectrum(L,B,xmin,xmax,ymin,ymax,Nxgrid,Nygrid,heights,fl,log(norm_V_random,10))
    
    
    
    # Eigenvalues
    eigenvalues_L, eigenvectors_L = LA.eig(L,B)
    #eigenvalues_L = LA.eigvals(L,B)
    #eigenvalues_L, eigenvectors_L = LA.eig(LTL_test)
    #eigenvalues_L *= -1j
    
    print("\n Eigenvalues = \n", eigenvalues_L)

    
    eigenvalues_L_Re = eigenvalues_L.real
    eigenvalues_L_Im = eigenvalues_L.imag

    #print("Re(s_n) = ", eigenvalues_L_Re )
    #print("Im(s_n) = ", eigenvalues_L_Im )

    
    return eigenvalues_L_Re, eigenvalues_L_Im, eigenvectors_L, x, xt

    #print(D1_CL(N))
    #print(D1_CL(N)[0])
    #print(D1_CL(N)[1])

    #x_CL, D1_CL = D1_CL(N)
    #D2_CL = D2_CL(N)[1]
    #print("Chebyshev-Lobatto grid points = ", x_CL)
    #print("D1_CL = ", D1_CL)
    #print("D2_CL = ", D2_CL)

## Graphical output of eigenvalues

The following is just a routine for plotting eigenvalues

In [10]:
def Eigenvalues_Graphic(Re_lambda, Im_lambda):
    # Graphical ouput

    eigenvalues_L_Re, eigenvalues_L_Im = Re_lambda, Im_lambda
    
    #print(eigenvalues_L_Re)
    #print(eigenvalues_L_Im)
    
    
    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111)

    ax1.plot(eigenvalues_L_Re, eigenvalues_L_Im, "+")
    ax1.set_xlabel(r'$\mathrm{Re}(\omega_n)$')
    ax1.set_ylabel(r'$\mathrm{Im}(\omega_n)$')  
    #ax1.set_aspect('equal','datalim')
    f = mticker.ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%10e' % x))
    ax1.xaxis.set_major_formatter(mticker.FuncFormatter(g))
    ax1.yaxis.set_major_formatter(mticker.FuncFormatter(g))
    
    fig1.suptitle(r'Eigenvalues of $L$ with $n=N+1=%5g$ points' %(N+1))
    ax1.grid()   
    #ax1.set_xlim(0.,500.)
    #ax1.set_ylim(-50.,50.)
    ax1.axis([-15,15,-12,20])

    fig1.show()
    #fig.savefig("/home/jaramillo/Dropbox/Trabajo/Programacion/Python/Diagonalization/Eigenvalues_L.pdf")
    #fig1.savefig("Eigenvalues_PT_Random1e-5_1.pdf")
    

## Main execution of the code

Arrays "a" and "b" contain the eigenvalues

In [11]:
N=25
n=1
a, b, c, x, xt = Matrix_Eigenvalues_Test(N, 1e-50)
Eigenvalues_Graphic(a,b)
vv = c[:,n]
v = vv[0:(N+1)]
vr = v.real
vi = v.imag
print("x=",x, "\n\n")
print("x0=",x[:,0], "\n\n")
print("c0=",c[:,n], "\n\n")
print("vr=",vr, "\n\n")
print("vi=",vi, "\n\n")

#plt.close()
fig2 = plt.figure()
ax2 = fig2.add_subplot(111)
ax2.plot(x[:,0],vr,"r-")
ax2.plot(x[:,0],vi,"b-")



NameError: name 'math' is not defined